In [1]:
import time
from pathlib import Path
import socket
from urllib.parse import urlparse
import requests
import pandas as pd

print('All imports are ready in one place.')

All imports are ready in one place.


# Step 1: Pre-check and Environment Setup

This cell checks for Docker, determines the correct `docker-compose` command, and verifies that no conflicting containers are running. This is a safety check to prevent errors when starting the services.

In [13]:
def is_port_open(host: str, port: int, timeout: float = 1.0) -> bool:
    """Check if a TCP port is open on the given host."""
    try:
        with socket.create_connection((host, port), timeout=timeout):
            return True
    except Exception:
        return False

def wait_for_port(url: str, timeout: int = 60, interval: float = 1.0) -> bool:
    """Wait until the host:port in the URL accepts TCP connections."""
    parsed = urlparse(url)
    host = parsed.hostname or 'localhost'
    port = parsed.port or (443 if parsed.scheme == 'https' else 80)
    start = time.time()
    while time.time() - start < timeout:
        if is_port_open(host, port, timeout=1.0):
            return True
        time.sleep(interval)
    return False


In [14]:
def perform_request_with_check(pattern, name, url, method='GET', payload=None, timeout=10):
    parsed = urlparse(url)
    host = parsed.hostname or 'localhost'
    port = parsed.port or (443 if parsed.scheme == 'https' else 80)

    # Measure wait duration separately
    wait_start = time.time()
    reachable = wait_for_port(url, timeout=60, interval=1.0)

    if not reachable:
        elapsed = time.time() - wait_start
        return {
            'pattern': pattern,
            'scenario': name,
            'url': url,
            'status_code': 'UNREACHABLE',
            'response_time': elapsed,
            'result': f'Host {host}:{port} not reachable within timeout'
        }

    # If reachable, measure HTTP request latency only
    req_start = time.time()
    try:
        if method.upper() == 'GET':
            r = requests.get(url, timeout=timeout)
        elif method.upper() == 'POST':
            r = requests.post(url, json=payload, timeout=timeout)
        else:
            raise ValueError(f"Unsupported method: {method}")
        elapsed = time.time() - req_start
        try:
            body = r.json()
        except Exception:
            body = r.text
        status_code = r.status_code
    except requests.exceptions.RequestException as e:
        elapsed = time.time() - req_start
        status_code = getattr(getattr(e, 'response', None), 'status_code', 'N/A')
        body = str(e)

    return {
        'pattern': pattern,
        'scenario': name,
        'url': url,
        'status_code': status_code,
        'response_time': elapsed,
        'result': body
    }

In [ ]:
# Health check endpoints
CHOREO_ORDER_HEALTH = 'http://localhost:8011/health'
ORCH_ORCHESTRATOR_HEALTH = 'http://localhost:8005/health'
ORCH_ORDER_HEALTH = 'http://localhost:8011/health'

# Define health check scenarios
health_checks = [
    ("choreography", "order-service", CHOREO_ORDER_HEALTH),
    ("orchestration", "saga-orchestrator", ORCH_ORCHESTRATOR_HEALTH),
    ("orchestration", "order-service", ORCH_ORDER_HEALTH),
]

In [ ]:
# Service endpoints (matching compose mappings)
CHOREO_ORDER_URL = 'http://localhost:8011/orders'      # choreography order-service
ORCH_ORCHESTRATOR_URL = 'http://localhost:8005/saga/start' # orchestrator entry (corrected)
ORCH_ORDER_HOST_PORT = 'http://localhost:8011/orders'  # orchestration's order-service mapped to 8011

# Output CSV path
out_csv = Path.cwd().parent / 'test_results.csv'  # saga_pattern/test_results.csv

scenarios = [
    ("choreography", "success", CHOREO_ORDER_URL, {
        "customer_id": "customer-001",
        "items": [{"book_id": "book-123", "quantity": 1}]
    }),
    ("choreography", "stock_failure", CHOREO_ORDER_URL, {
        "customer_id": "customer-002",
        "items": [{"book_id": "book-456", "quantity": 1}]
    }),
    ("choreography", "payment_failure", CHOREO_ORDER_URL, {
        "customer_id": "customer-004",
        "items": [{"book_id": "book-789", "quantity": 1}]
    }),
    ("orchestration", "orchestrator_submit", ORCH_ORCHESTRATOR_URL, {
        "customer_id": "customer-001",
        "items": [{"book_id": "book-123", "quantity": 1}]
    }),
    ("orchestration", "order_service_direct", ORCH_ORDER_HOST_PORT, {
        "customer_id": "customer-001",
        "items": [{"book_id": "book-123", "quantity": 1}]
    }),
]

In [ ]:
print("=== Health Check Results ===")
for pattern, service_name, health_url in health_checks:
    result = perform_request_with_check(pattern, service_name, health_url, method='GET')

    if result['status_code'] == 'UNREACHABLE':
        status = f"✗ {result['result']}"
    else:
        status = f"✓ {result['status_code']}"
        if result['status_code'] == 200:
            try:
                response_data = result['result']
                if isinstance(response_data, dict):
                    status += f" - {response_data.get('status', 'OK')}"
                else:
                    status += " - OK"
            except Exception as e:
                status += " - OK"
                print(f"Warning: Failed to parse JSON response from {service_name}: {e}")
        else:
            status += f" - {str(result['result'])[:50]}..."

    print(f"{pattern:12} | {service_name:15} | {health_url:30} | {status}")

print("\n" + "="*80)

=== Health Check Results ===
choreography | order-service   | http://localhost:8011/health   | ✓ 200 - healthy
orchestration | saga-orchestrator | http://localhost:8005/health   | ✓ 200 - healthy
orchestration | order-service   | http://localhost:8011/health   | ✓ 200 - healthy



In [ ]:
print("\n=== Running Saga Pattern Tests ===")
results = []

# Run scenarios with port wait and unreachable handling
for pattern, name, url, payload in scenarios:
    result = perform_request_with_check(pattern, name, url, method='POST', payload=payload)
    results.append(result)

# Build DataFrame and save CSV
expected_cols = ['pattern', 'scenario', 'url', 'status_code', 'response_time', 'result']
df = pd.DataFrame(results, columns=expected_cols)
df['result'] = df['result'].apply(lambda x: str(x) if x is not None else '')
out_csv.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_csv, index=False)
print('Wrote test results to', out_csv)
print(df)


=== Running Saga Pattern Tests ===
Wrote test results to /Users/codefox/workspace/practice_infra_arch/test_results.csv
         pattern              scenario                               url  \
0   choreography               success      http://localhost:8011/orders   
1   choreography         stock_failure      http://localhost:8011/orders   
2   choreography       payment_failure      http://localhost:8011/orders   
3  orchestration   orchestrator_submit  http://localhost:8005/saga/start   
4  orchestration  order_service_direct      http://localhost:8011/orders   

   status_code  response_time  \
0          200       0.044899   
1          200       0.012511   
2          200       0.010876   
3          200       0.020904   
4          200       0.006526   

                                              result  
0  {'order_id': 'order-eb13f20308b54651a4a60fb286...  
1  {'order_id': 'order-44838860df544e2abdd9c85aaf...  
2  {'order_id': 'order-0eeb2628dfd040cf9b00d082e5...  
3  {

# Step 3: Run Smoke Tests

This cell runs a series of smoke tests against both the choreography and orchestration patterns. It checks for service availability and then sends test requests to simulate different scenarios (success, stock failure, payment failure). The results are collected and saved to a CSV file for analysis.